# Objective

### Develop a classifier to predict pitch type suitable for returning predictions in real-time.

In [1]:
import itertools
import warnings

import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk

# helper methods for the notebook
import classifier as cl

warnings.filterwarnings('ignore')
plt.style.use('ggplot')

# Understanding the data and cleaning

In [2]:
# load only meaningful features available at pitch time and target label
uninformative_feature_set = set([
    'uid', 'game_pk', 'year', 'start_tfs', 'start_tfs_zulu', 'pitch_id'])
target_label = 'pitch_type'
features_desc = cl.get_feature_set_descriptions(
    uninformative_feature_set, target_label)

### Peeking at the feature metadata, it looks like a good first pass would be to use only features available at the time of the pitch, and ignore seemingly uninformative identifiers

In [3]:
train = pd.read_csv(
    '../../data/pitches.csv',
    usecols=features_desc.index, parse_dates=['date'])

In [4]:
print(train.shape)
display(train.head())
display(train.dtypes)
display(train.pitch_type.value_counts())
print(f'missing label count is {train.shape[0] - train.pitch_type.value_counts().sum()}')

(718961, 23)


,date,team_id_b,team_id_p,inning,top,at_bat_num,pcount_at_bat,pcount_pitcher,balls,strikes,...,stand,b_height,pitcher_id,p_throws,away_team_runs,home_team_runs,pitch_type,on_1b,on_2b,on_3b
0,2011-03-31,108,118,1,1,1,1,1,0,0,...,L,5-8,460024,R,0,0,NaN,NaN,NaN,NaN
1,2011-03-31,108,118,1,1,1,2,2,1,0,...,L,5-8,460024,R,0,0,NaN,NaN,NaN,NaN
2,2011-03-31,108,118,1,1,1,3,3,2,0,...,L,5-8,460024,R,0,0,NaN,NaN,NaN,NaN
3,2011-03-31,108,118,1,1,1,4,4,2,1,...,L,5-8,460024,R,0,0,NaN,NaN,NaN,NaN
4,2011-03-31,108,118,1,1,2,1,5,0,0,...,R,5-10,460024,R,0,0,NaN,NaN,NaN,NaN


date              datetime64[ns]
team_id_b                  int64
team_id_p                  int64
inning                     int64
top                        int64
at_bat_num                 int64
pcount_at_bat              int64
pcount_pitcher             int64
balls                      int64
strikes                    int64
fouls                      int64
outs                       int64
batter_id                  int64
stand                     object
b_height                  object
pitcher_id                 int64
p_throws                  object
away_team_runs             int64
home_team_runs             int64
pitch_type                object
on_1b                    float64
on_2b                    float64
on_3b                    float64
dtype: object

FF    238541
SL    109756
SI     87740
FT     81056
CH     72641
CU     56379
FC     41702
FS     10503
KC      8490
KN      4450
IN      4058
PO       559
FO       329
FA       204
EP       134
SC       120
UN        17
AB         2
Name: pitch_type, dtype: int64

missing label count is 2280


### The raw read has misinterpreted some features as numeric when they should be categories, which should be fixed.

### The output classes are heavily imbalanced, and I think a benchmark will perform at about 30% accuracy just predicting pitch_type='FF'

In [5]:
numerical_features = [
    'at_bat_num', 'away_team_runs', 'b_height', 'balls', 'fouls',
    'home_team_runs', 'inning', 'outs', 'pcount_at_bat',
    'pcount_pitcher', 'strikes', 'top']
float_to_int_features = ['on_1b', 'on_2b', 'on_3b']
categorical_features = [
    'batter_id', 'on_1b', 'on_2b', 'on_3b', 'pitcher_id',
    'p_throws', 'stand', 'team_id_b', 'team_id_p']
target_label = 'pitch_type'
feature_type_map = {
    'categorical': categorical_features, 
    'numerical': numerical_features,
    'float_to_int': float_to_int_features,
    'target': target_label}

In [6]:
cl.clean_pitches(train, feature_type_map)

In [7]:
display(train.head())
display(train.dtypes)
train.isnull().sum().sum()

,date,team_id_b,team_id_p,inning,top,at_bat_num,pcount_at_bat,pcount_pitcher,balls,strikes,...,stand,b_height,pitcher_id,p_throws,away_team_runs,home_team_runs,pitch_type,on_1b,on_2b,on_3b
0,2011-03-31,108,118,1,1,1,1,1,0,0,...,L,21,460024,R,0,0,missing,0,0,0
1,2011-03-31,108,118,1,1,1,2,2,1,0,...,L,21,460024,R,0,0,missing,0,0,0
2,2011-03-31,108,118,1,1,1,3,3,2,0,...,L,21,460024,R,0,0,missing,0,0,0
3,2011-03-31,108,118,1,1,1,4,4,2,1,...,L,21,460024,R,0,0,missing,0,0,0
4,2011-03-31,108,118,1,1,2,1,5,0,0,...,R,26,460024,R,0,0,missing,0,0,0


date              datetime64[ns]
team_id_b               category
team_id_p               category
inning                     int64
top                        int64
at_bat_num                 int64
pcount_at_bat              int64
pcount_pitcher             int64
balls                      int64
strikes                    int64
fouls                      int64
outs                       int64
batter_id               category
stand                   category
b_height                   int64
pitcher_id              category
p_throws                category
away_team_runs             int64
home_team_runs             int64
pitch_type              category
on_1b                   category
on_2b                   category
on_3b                   category
dtype: object

0

### The cleaned features use categorical with empty/NaN modeled as 'missing' and numerical with empty/NaN modeled as 0. There are nuances to 'missing' and 0 that are not entirely accurate but are good enough for now.

# Transforming the data for modeling

In [8]:
# split the training data for testing by the date at which 80% of pitches are thrown
X_calval, X_holdout, y_calval, y_holdout = cl.serial_calval_holdout_split(train)

### I split the data by date to guard against data leakage for the holdout data set

In [9]:
display(X_calval.shape)
display(X_holdout.shape)
display(X_holdout.dtypes)

(574882, 21)

(144079, 21)

team_id_b         category
team_id_p         category
inning               int64
top                  int64
at_bat_num           int64
pcount_at_bat        int64
pcount_pitcher       int64
balls                int64
strikes              int64
fouls                int64
outs                 int64
batter_id         category
stand             category
b_height             int64
pitcher_id        category
p_throws          category
away_team_runs       int64
home_team_runs       int64
on_1b             category
on_2b             category
on_3b             category
dtype: object

In [10]:
ct = cl.get_column_transformer(feature_type_map)
ct.fit(X_calval)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('cat',
                                 OneHotEncoder(categorical_features=None,
                                               categories=None, drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='ignore',
                                               n_values=None, sparse=True),
                                 ['batter_id', 'on_1b', 'on_2b', 'on_3b',
                                  'pitcher_id', 'p_throws', 'stand',
                                  'team_id_b', 'team_id_p']),
                                ('num', PassthroughTransformer(),
                                 ['at_bat_num', 'away_team_runs', 'b_height',
                                  'balls', 'fouls', 'home_team_runs', 'inning',
                                  'outs',

### The column transformer is fitted only on "calval" to avoid data leakage. This custom transformer ensures feature names are available for this mix of numerical and categorical features.

In [11]:
X_trans_calval = ct.transform(X_calval)
X_trans_holdout = ct.transform(X_holdout)
trans_feature_names = cl.get_feature_names(ct, feature_type_map)

In [12]:
display(X_trans_calval.shape)
display(X_trans_holdout.shape)
display(trans_feature_names[-20:-1])

(574882, 3648)

(144079, 3648)

['team_id_p_141',
 'team_id_p_142',
 'team_id_p_143',
 'team_id_p_144',
 'team_id_p_145',
 'team_id_p_146',
 'team_id_p_147',
 'team_id_p_158',
 'at_bat_num',
 'away_team_runs',
 'b_height',
 'balls',
 'fouls',
 'home_team_runs',
 'inning',
 'outs',
 'pcount_at_bat',
 'pcount_pitcher',
 'strikes']

# Classification model R&D

In [13]:
from time import time

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [14]:
sample_size = int(5e4)
random_row_indices = np.random.choice(X_trans_calval.shape[0], size=sample_size, replace=False)
X_trans_calval_sample = X_trans_calval[random_row_indices,]
y_calval_sample = y_calval[random_row_indices]

### Some of the models I chose struggle with a data set this large on my laptop, so I took a random sample of rows of about 10% of the data set.

In [15]:
clf = DummyClassifier(strategy='most_frequent')
start = time()
clf.fit(X_trans_calval, y_calval)
print(f"Fit took {time() - start:.2f} seconds.")

accuracy = accuracy_score(y_calval, clf.predict(X_trans_calval))
print(f"sample calval accuracy is {accuracy:.2f}.")
accuracy = accuracy_score(y_holdout, clf.predict(X_trans_holdout))
print(f"holdout accuracy is {accuracy:.2f}.")

Fit took 0.75 seconds.
sample calval accuracy is 0.33.
holdout accuracy is 0.34.


### As expected, a dummy classifier achieves 33% accuracy, so this is the benchmark to beat.

In [16]:
clf = RandomForestClassifier(n_jobs=-1)
start = time()
clf.fit(X_trans_calval_sample, y_calval_sample)
print(f"Fit took {time() - start:.2f} seconds.")

accuracy = accuracy_score(y_calval_sample, clf.predict(X_trans_calval_sample))
print(f"sample calval accuracy is {accuracy:.2f}.")
accuracy = accuracy_score(y_holdout, clf.predict(X_trans_holdout))
print(f"holdout accuracy is {accuracy:.2f}.")
print(f"most important features are:")
display(np.array(trans_feature_names)[list(clf.feature_importances_>0.01)])

Fit took 15.37 seconds.
sample calval accuracy is 0.98.
holdout accuracy is 0.38.
most important features are:


array(['on_1b_0', 'at_bat_num', 'away_team_runs', 'b_height', 'balls',
       'fouls', 'home_team_runs', 'inning', 'outs', 'pcount_at_bat',
       'pcount_pitcher', 'strikes', 'top'], dtype='<U17')

### A basic random forest overfits to the calval data set but achieves 38% on the holdout, so about 15% lift. Not bad for out the box settings on such a small sample.

In [17]:
clf = LogisticRegression(solver='liblinear', multi_class='ovr')
start = time()
clf.fit(X_trans_calval_sample, y_calval_sample)
print(f"Fit took {time() - start:.2f} seconds.")

accuracy = accuracy_score(y_calval_sample, clf.predict(X_trans_calval_sample))
print(f"sample calval accuracy is {accuracy:.2f}.")
accuracy = accuracy_score(y_holdout, clf.predict(X_trans_holdout))
print(f"holdout accuracy is {accuracy:.2f}.")

Fit took 97.11 seconds.
sample calval accuracy is 0.54.
holdout accuracy is 0.44.


### A basic logistic regression with some regularization overfits less than the random forest and get 44% accuracy, so about 33% lift. I'll investigate this model further.

In [18]:
clf = LogisticRegression(solver='liblinear', multi_class='ovr')
param_grid = {'C': [0.1, 1, 10]}
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=2)
start = time()
grid_search.fit(X_trans_calval_sample, y_calval_sample)
n_param_sets = len(grid_search.cv_results_['params'])
print(f"GridSearchCV took {time() - start:.2f} seconds for {n_param_sets:d} candidate parameter settings.")
display(grid_search.cv_results_)

GridSearchCV took 364.97 seconds for 3 candidate parameter settings.


{'mean_fit_time': array([24.64520156, 42.75359654, 65.32481992]),
 'std_fit_time': array([0.31587851, 0.03793955, 1.14612997]),
 'mean_score_time': array([0.07792902, 0.0768826 , 0.07793546]),
 'std_score_time': array([0.00181508, 0.00072241, 0.00463152]),
 'param_C': masked_array(data=[0.1, 1, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.1}, {'C': 1}, {'C': 10}],
 'split0_test_score': array([0.43411318, 0.44679064, 0.42839432]),
 'split1_test_score': array([0.43356671, 0.44964993, 0.43316663]),
 'mean_test_score': array([0.43384, 0.44822, 0.43078]),
 'std_test_score': array([0.00027323, 0.00142964, 0.00238616]),
 'rank_test_score': array([2, 1, 3], dtype=int32)}

### Grid search on the regularization hyperparameter and twofold cross validation show consistent results with the out of the box settings for Logistic Regression and yield accuracy of 45%, consistent with the prior experiment.

### The model has thousands of features and does not scale well, so I will retrain the model using only the most important features using all the calval data set.

In [19]:
sfm = SelectFromModel(estimator=grid_search.best_estimator_, threshold='mean', max_features=100)
start = time()
sfm.fit(X_trans_calval_sample, y_calval_sample)
print(f"Fit took {time() - start:.2f} seconds.")
X_trans_calval_subset_features = sfm.transform(X_trans_calval)
X_trans_holdout_subset_features = sfm.transform(X_trans_holdout)
print(X_trans_calval_subset_features.shape)

clf = LogisticRegression(solver='liblinear', multi_class='ovr')
start = time()
clf.fit(X_trans_calval_subset_features, y_calval)
print(f"Fit took {time() - start:.2f} seconds.")

accuracy = accuracy_score(y_calval, clf.predict(X_trans_calval_subset_features))
print(f"calval accuracy is {accuracy:.2f}.")
accuracy = accuracy_score(y_holdout, clf.predict(X_trans_holdout_subset_features))
print(f"holdout accuracy is {accuracy:.2f}.")

Fit took 102.61 seconds.
(574882, 100)
Fit took 124.76 seconds.
calval accuracy is 0.41.
holdout accuracy is 0.40.


### Training logistic regression using only 100 of the transformed features results in holdout accuracy of 40% and lift of 20%.

In [20]:
sfm = SelectFromModel(estimator=grid_search.best_estimator_, threshold='mean', max_features=300)
start = time()
sfm.fit(X_trans_calval_sample, y_calval_sample)
print(f"Fit took {time() - start:.2f} seconds.")
X_trans_calval_subset_features = sfm.transform(X_trans_calval)
X_trans_holdout_subset_features = sfm.transform(X_trans_holdout)
print(X_trans_calval_subset_features.shape)

clf = LogisticRegression(solver='liblinear', multi_class='ovr')
start = time()
clf.fit(X_trans_calval_subset_features, y_calval)
print(f"Fit took {time() - start:.2f} seconds.")

accuracy = accuracy_score(y_calval, clf.predict(X_trans_calval_subset_features))
print(f"calval accuracy is {accuracy:.2f}.")
accuracy = accuracy_score(y_holdout, clf.predict(X_trans_holdout_subset_features))
print(f"holdout accuracy is {accuracy:.2f}.")

Fit took 130.91 seconds.
(574882, 300)
Fit took 407.06 seconds.
calval accuracy is 0.46.
holdout accuracy is 0.44.


### Training logistic regression using only 300 of the transformed features results in holdout accuracy of 44% and lift of 33%.

# Conclusion

### Model selection should balance the need for model performance and computational resource requirements, along with deployability. For this reason, I propose the best model is the Logistic Regression trained using only the top 300 features and would deploy this fitted model. 

### Balancing the trade-offs between development time, training time, and other performance metrics would require further economic data such as cost of inaccurate predictions and entry to market delays.

In [21]:
import pickle

# save the settings used to pickle file for use in deploy
model_metadata = {
    'hyperparameters': {'C': 1},
    'sample_size': int(5e4),
    'max_features': 300
}
with open('model_metadata.pkl', 'wb') as fh:
    pickle.dump(model_metadata, fh, protocol=pickle.HIGHEST_PROTOCOL)